In [48]:
import pandas as pd
from utils import wcc
from utils import process_dataframe
from utils import function

In [49]:
def concatenar_coluna_para_string(df, nome_coluna):
    # Verificar se a coluna existe no dataframe
    if nome_coluna not in df.columns:
        return "Coluna não encontrada."

    # Concatenar todos os itens da coluna em uma string
    string_concatenada = ''.join(df[nome_coluna].astype(str))

    return string_concatenada

In [37]:
def igualar_tamanho(df1, df2):
    if len(df1) > len(df2):
        df1=df1.head(-1)
    elif len(df2) > len(df1):
        df2=df2.head(-1)
    else:
        a=1
    return df1, df2

In [19]:
df=pd.read_excel("/home/lucas/Documentos/SentimentAnalysisInMentalHealth/Datasets/transcribe_sentiment_classified.xlsx")

In [21]:
simbols=[]
for index,row in df.iterrows():
    if row["Sentimento"]=="neutro":
        simbols.append('n')
    elif row["Sentimento"]=="positivo":
        simbols.append('p')
    else:
        simbols.append('z')

In [24]:
df["simbol_sentiment"]=simbols

In [42]:
####
#df.sessao.unique()
df_sessao=df.loc[df["sessao"]== 'SPCOMVC_T010_P001_09052020']
sumarizacao_sessao=process_dataframe.processar_dataframe(df_sessao,"Falante","simbol_sentiment")
df_paciente=sumarizacao_sessao[sumarizacao_sessao["Categoria"]=="p"]
df_terapeuta=sumarizacao_sessao[sumarizacao_sessao["Categoria"]=="t"]
df_paciente,df_terapeuta=igualar_tamanho(df_paciente,df_terapeuta)

In [51]:
sumarizacao_sessao

,Categoria,Primeiro_valor_B,Ultimo_valor_B,Mais_repetido_B,inicio
0,t,n,n,n,0.0
0,p,n,z,z,3.0
0,t,n,n,n,10.0
0,p,n,n,n,13.0
0,t,n,n,n,15.0
...,...,...,...,...,...
0,t,n,n,n,1181.0
0,p,p,p,p,1183.0
0,t,p,n,n,1184.0
0,p,n,n,n,1187.0


In [46]:
string_paciente=concatenar_coluna_para_string(df_paciente, "Primeiro_valor_B")
string_terapeuta=concatenar_coluna_para_string(df_terapeuta, "Primeiro_valor_B")

In [53]:
function.needleman_wunsch(string_paciente, string_terapeuta, match_score=10, mismatch_score=-10, gap_penalty=-2)

(674,
 '-nn--nnpnnnnz-------nnnpznppnz--nnznnnnnznznnnnzn-nnnnznnnnpnnnnpnnnnnnnnznpnnznnnznnnnnnnnnnznnn-npn',
 'nnnzznn-nnnnznnnnnnnnnn-zn--nznnnn-nnnnnzn-nnnn-npnnnn-nnnn-nnnn-nnnnnnnn-n-nnznnnznnnnnnnnnn-nnnpnp-')

In [57]:
df_output=pd.DataFrame(columns=['sessao', 'itens_paciente', 'itens_terapeuta','WaiPaciente','WaiTerapeuta','score'])
for sessao in df.sessao.unique():
    df_sessao=df.loc[df["sessao"]== sessao]
    sentimentos=["p","n","z"]
    combinacoes_sentimento = [(sentimento_p, sentimento_t) for sentimento_p in ["positivo","negativo","neutro"] 
                                       for sentimento_t in ["positivo","negativo","neutro"] ]
    sumarizacao_sessao=process_dataframe.processar_dataframe(df_sessao,"Falante","simbol_sentiment")
    df_paciente=sumarizacao_sessao[sumarizacao_sessao["Categoria"]=="p"]
    df_terapeuta=sumarizacao_sessao[sumarizacao_sessao["Categoria"]=="t"]
    df_paciente,df_terapeuta=igualar_tamanho(df_paciente,df_terapeuta)
    combinacoes_itens = [(p, t) for p in ["Primeiro_valor_B", "Ultimo_valor_B", "Media_B"] 
                                       for t in ["Primeiro_valor_B", "Ultimo_valor_B", "Media_B"]]
        
        # Loop sobre as combinações de itens_paciente e itens_terapeuta
    for itens_paciente, itens_terapeuta in combinacoes_itens:
        string_paciente=concatenar_coluna_para_string(df_paciente, itens_paciente)
        string_terapeuta=concatenar_coluna_para_string(df_terapeuta, itens_terapeuta)
        score,x1,x2=function.needleman_wunsch(string_paciente, string_terapeuta, match_score=1, mismatch_score=-1, gap_penalty=-1)
        df_output = pd.concat([df_output,pd.DataFrame.from_dict({'sessao': [sessao], 
                                                                             'itens_paciente': [itens_paciente], 
                                                                             'itens_terapeuta': [itens_terapeuta], 
                                                                             'WaiPaciente':[df_sessao["WAI-SR-P TOTAL"].values[0]],
                                                                             'WaiTerapeuta':[df_sessao["WAI-SR-T TOTAL"].values[0]],
                                                                             'score':[score]
                                                                            })])

/tmp/ipykernel_1402556/2623530329.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_output = pd.concat([df_output,pd.DataFrame.from_dict({'sessao': [sessao],


In [59]:
df_output.to_excel("result_NW.xlsx")

In [60]:
df_result=pd.read_excel("result_NW2.xlsx")

In [61]:
df_result.head()

,sessao,itens_paciente,itens_terapeuta,WaiPaciente,WaiTerapeuta,score,Comb
0,SPCOMVC_T010_P001_09052020,Primeiro_valor_B,Media_B,38.0,38.0,-79,3.0
1,SPCOMVC_T001_P002_11052020,Primeiro_valor_B,Media_B,42.0,42.0,-236,3.0
2,SPCOMVC_T001_P002_11062020,Primeiro_valor_B,Media_B,50.0,50.0,-120,3.0
3,SPCOMVC_T001_P002_18052020,Primeiro_valor_B,Media_B,45.0,45.0,-343,3.0
4,SPCOMVC_T001_P002_24062020,Primeiro_valor_B,Media_B,49.0,49.0,-192,3.0


In [64]:
correl_list_paciente=[]
correl_list_terapeuta=[]
for comb in df_result.Comb.unique():
    df_result_comb=df_result[df_result["Comb"]==comb]
    #for index,row in df_result_comb.iterrows():
    #    if not np.isnotnan(row["max"]) and not np.isnan(row["max"]):
    df_filtrado_T = df_result_comb.dropna(subset=['WaiTerapeuta'])
    df_filtrado_P = df_result_comb.dropna(subset=['WaiPaciente'])
    correl_list_terapeuta.append(df_filtrado_T['score'].corr(df_filtrado_T['WaiTerapeuta']))
    correl_list_paciente.append(df_filtrado_P['score'].corr(df_filtrado_P['WaiPaciente']))
    print(comb,df_filtrado_P['score'].corr(df_filtrado_T['WaiPaciente']))

3.0 0.0923867159389665
1.0 -0.14750348868847277
2.0 -0.20839223343950172
6.0 0.09350972529632767
4.0 -0.15323109924819356
5.0 -0.20734753927338878
9.0 0.11893653420753485
7.0 0.10962907033658507
8.0 0.12409213565002158
nan nan
